In [5]:
!pip install sqlalchemy

import sqlalchemy
sqlalchemy.__version__

import urllib.request
import zipfile
from functools import partial
import os

!unzip /content/chinook.zip -d /content

Archive:  /content/chinook.zip
  inflating: /content/chinook.db     


In [6]:
from IPython.display import display
import pandas as pd

def sql(query):
    print()
    print(query)
    print()

def get_results(query):
    global engine
    q = query.statement if isinstance(query, sqlalchemy.orm.query.Query) else query
    return pd.read_sql(q, engine)

def display_results(query):
    df = get_results(query)
    display(df)
    sql(query)

In [13]:
# Database file path
db_file = '/content/chinook.db'

# Create an engine object
engine = sqlalchemy.create_engine(f'sqlite:///{db_file}')
conn = engine.connect()

metadata = sqlalchemy.MetaData()
metadata.reflect(engine)

## we need to do this once
from sqlalchemy.ext.automap import automap_base

# produce a set of mappings from this MetaData.
Base = automap_base(metadata=metadata)

# calling prepare() just sets up mapped classes and relationships.
Base.prepare()

# also prepare an orm session
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

In [14]:
#Exercise 2
inspector = sqlalchemy.inspect(engine)
table_names = inspector.get_table_names()
print(table_names)

#Exercise3
query = """
SELECT *
FROM tracks
LIMIT 3
"""
get_results(query)
display_results(query)

['albums', 'artists', 'customers', 'employees', 'genres', 'invoice_items', 'invoices', 'media_types', 'playlist_track', 'playlists', 'tracks']


,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99




SELECT * 
FROM tracks
LIMIT 3




In [15]:
#exercise4
query = """
SELECT
    T.Name AS TrackName,
    A.Title AS AlbumTitle
FROM
    tracks AS T
JOIN
    albums AS A ON T.AlbumId = A.AlbumId
LIMIT 20
"""
get_results(query)
display_results(query)

,TrackName,AlbumTitle
0,For Those About To Rock (We Salute You),For Those About To Rock We Salute You
1,Put The Finger On You,For Those About To Rock We Salute You
2,Let's Get It Up,For Those About To Rock We Salute You
3,Inject The Venom,For Those About To Rock We Salute You
4,Snowballed,For Those About To Rock We Salute You
5,Evil Walks,For Those About To Rock We Salute You
6,C.O.D.,For Those About To Rock We Salute You
7,Breaking The Rules,For Those About To Rock We Salute You
8,Night Of The Long Knives,For Those About To Rock We Salute You
9,Spellbound,For Those About To Rock We Salute You




SELECT 
    T.Name AS TrackName,
    A.Title AS AlbumTitle
FROM 
    tracks AS T
JOIN 
    albums AS A ON T.AlbumId = A.AlbumId
LIMIT 20




In [16]:
#exercise5
query = """
SELECT
    T.Name AS TrackName,
    II.Quantity
FROM
    invoice_items AS II
JOIN
    tracks AS T ON II.TrackId = T.TrackId
LIMIT 10
"""
get_results(query)
display_results(query)

,TrackName,Quantity
0,Balls to the Wall,1
1,Restless and Wild,1
2,Put The Finger On You,1
3,Inject The Venom,1
4,Evil Walks,1
5,Breaking The Rules,1
6,Dog Eat Dog,1
7,Overdose,1
8,Love In An Elevator,1
9,Janie's Got A Gun,1




SELECT 
    T.Name AS TrackName,
    II.Quantity
FROM 
    invoice_items AS II
JOIN 
    tracks AS T ON II.TrackId = T.TrackId
LIMIT 10




In [17]:
#exercise 6
query = """
SELECT
    T.Name AS TrackName,
    SUM(II.Quantity) AS TotalSold
FROM
    invoice_items AS II
JOIN
    tracks AS T ON II.TrackId = T.TrackId
GROUP BY
    T.Name
ORDER BY
    TotalSold DESC
LIMIT 10
"""

get_results(query)
display_results(query)

,TrackName,TotalSold
0,The Trooper,5
1,Untitled,4
2,The Number Of The Beast,4
3,Sure Know Something,4
4,Hallowed Be Thy Name,4
5,Eruption,4
6,Where Eagles Dare,3
7,Welcome Home (Sanitarium),3
8,Sweetest Thing,3
9,Surrender,3




SELECT 
    T.Name AS TrackName,
    SUM(II.Quantity) AS TotalSold
FROM 
    invoice_items AS II
JOIN 
    tracks AS T ON II.TrackId = T.TrackId
GROUP BY 
    T.Name
ORDER BY 
    TotalSold DESC
LIMIT 10




In [18]:
#exercise 7
query = """
SELECT
    AR.Name AS ArtistName,
    SUM(II.UnitPrice * II.Quantity) AS TotalSalesValue
FROM
    invoice_items AS II
JOIN
    tracks AS T ON II.TrackId = T.TrackId
JOIN
    albums AS AL ON T.AlbumId = AL.AlbumId
JOIN
    artists AS AR ON AL.ArtistId = AR.ArtistId
GROUP BY
    AR.Name
ORDER BY
    TotalSalesValue DESC
LIMIT 10;
"""

get_results(query)
display_results(query)

,ArtistName,TotalSalesValue
0,Iron Maiden,138.60
1,U2,105.93
2,Metallica,90.09
3,Led Zeppelin,86.13
4,Lost,81.59
5,The Office,49.75
6,Os Paralamas Do Sucesso,44.55
7,Deep Purple,43.56
8,Faith No More,41.58
9,Eric Clapton,39.60




SELECT 
    AR.Name AS ArtistName,
    SUM(II.UnitPrice * II.Quantity) AS TotalSalesValue
FROM 
    invoice_items AS II
JOIN 
    tracks AS T ON II.TrackId = T.TrackId
JOIN 
    albums AS AL ON T.AlbumId = AL.AlbumId
JOIN 
    artists AS AR ON AL.ArtistId = AR.ArtistId
GROUP BY 
    AR.Name
ORDER BY 
    TotalSalesValue DESC
LIMIT 10;


